# Assignment 2

## Part 1 (Evaluating RBF Kernels)

Used Packages for this assignment:

In [1]:
from numba import cuda
from matplotlib.colors import LogNorm
from matplotlib import pyplot as plt
import numpy as np
import numba
import math
from scipy.sparse import coo_matrix

### CPU Implementation

In the lectures we have used a rbf evaluation using the CPU. We will use the code below for validating my solution and also benchmarking it:

In [2]:
sigma = .1
npoints = 400
nsources = 64

plot_grid = np.mgrid[0:1:npoints * 1j, 0:1:npoints * 1j]

targets_xy = np.vstack((plot_grid[0].ravel(),
                        plot_grid[1].ravel(),
                        np.zeros(plot_grid[0].size))).T
targets_xz = np.vstack((plot_grid[0].ravel(),
                        np.zeros(plot_grid[0].size),
                        plot_grid[1].ravel())).T
targets_yz = np.vstack((np.zeros(plot_grid[0].size),
                        plot_grid[0].ravel(),
                        plot_grid[1].ravel())).T

targets = np.vstack((targets_xy, targets_xz, targets_yz))
result_cpu = np.zeros(len(targets), dtype=np.float64)
rand = np.random.RandomState(0)

# We are picking random sources,

sources = rand.rand(nsources, 3)
weights = rand.rand(len(sources))

@numba.njit(parallel=True)
def rbf_evaluation_cpu(sources, targets, weights, result):
    """Evaluate the RBF sum."""

    n = len(sources)
    m = len(targets)

    result[:] = 0
    for index in numba.prange(m):
        result[index] = np.sum(
            np.exp(-np.sum(np.abs(targets[index] - sources)**2, axis=1) / (2 * sigma**2)) * weights)
    
rbf_evaluation_cpu(sources, targets, weights, result_cpu)


### GPU Implementation
The code block below is the same implementation as above but we are using GPU Cuda acceleration:

In [3]:
#16 targets and 32 sources
SX = 16
SY = 32

#Launch threadblocks that evaluate the partial sum for 16 targets and 32 sources at a time
@cuda.jit()
def rbf_evaluation_cuda(sources, targets, weights, inter_results):
    #local results only loads 32 sources for each block
    local_result = cuda.shared.array((SX, SY), numba.float32)
    local_targets = cuda.shared.array((SX, 3), numba.float32)
    local_sources = cuda.shared.array((SY, 3), numba.float32)
    local_weights = cuda.shared.array(SY, numba.float32)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    px, py = cuda.grid(2)
    
    #checking boundaries to only process valid px,py
    if px < targets.shape[0] or py < sources.shape[0]:

        # At first we are loading all the targets into the shared memory
        # We use only the first column of threads to do this.
        if ty == 0:
            for index in range(3):
                local_targets[tx, index] = targets[px, index]

        # We are now loading all the sources and weights.
        # We only require the first row of threads to do this.
        if tx == 0:
            for index in range(3):
                local_sources[ty, index] = sources[py, index]
            # up to 32 weights
            local_weights[ty] = weights[py]

        # Let us now sync all threads

        cuda.syncthreads()

        # Now compute the interactions

        squared_diff = numba.float32(0)

        for index in range(3):
            squared_diff += (local_targets[tx, index] -
                             local_sources[ty, index])**2
        local_result[tx, ty] = math.exp(-squared_diff / (
            numba.float32(2) * numba.float32(sigma)**2)) * local_weights[ty]

        cuda.syncthreads()
        

        # Now sum up all the local results
        if ty == 0:
            res = numba.float32(0)
            for index in range(SY):
                res += local_result[tx, index]
            # intermediate results that is of size (m, p).    
            inter_results[px, cuda.blockIdx.y] = res

#launch a summation kernel of m threads, where each thread sums up the p numbers 
@cuda.jit()
def summation_kernel(inter_results, results, targets):
    px, py = cuda.grid(2)
    ty = cuda.threadIdx.y
    
    #checking boundaries for safety, would not be a big deal since the last block,
    # we would be summing zero inter. values
    if px < targets.shape[0]:
        res = numba.float32(0)
        for index in range(p):
            #intermediate array (m, p) 
            res += inter_results[px, index]
            #store into a result array of dimension m
            results[px] = res


            
#blocks per grid
l = (targets.shape[0] + SX - 1) // SX
p = (sources.shape[0] + SY - 1) // SY

#CUDA memory transfer functions to copy the sources and targets to the compute device
sources_global_mem = cuda.to_device(sources.astype('float32'))
targets_global_mem = cuda.to_device(targets.astype('float32'))
weights_global_mem = cuda.to_device(weights.astype('float32'))
inter_result_global_mem = cuda.device_array((len(targets), p), dtype=np.float32)
result_global_mem = cuda.device_array(len(targets), dtype=np.float32)

rbf_evaluation_cuda[(l, p), (SX, SY)](
        sources_global_mem, targets_global_mem, weights_global_mem, inter_result_global_mem)

#Got the intermediate results,
#We do not want to transfer the intermediate array back and forth between CPU and GPU

#summation kernel, l * SX gets all targets and sums inter.results p
summation_kernel[(l, 1), (SX, 1)](inter_result_global_mem, result_global_mem,targets_global_mem)

#now we can copy the memory back to the host
result_gpu = result_global_mem.copy_to_host()

### Validation: 

In [4]:
def visualize(result, npoints):
    """A helper function for visualization"""

    result_xy = result[: npoints * npoints].reshape(npoints, npoints).T
    result_xz = result[npoints * npoints: 2 *
                       npoints * npoints].reshape(npoints, npoints).T
    result_yz = result[2 * npoints * npoints:].reshape(npoints, npoints).T

    fig = plt.figure(figsize=(20, 20))

    ax = fig.add_subplot(1, 3, 1)
    im = ax.imshow(result_xy, extent=[0, 1, 0, 1], origin='lower')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    ax = fig.add_subplot(1, 3, 2)
    im = ax.imshow(result_xz, extent=[0, 1, 0, 1], origin='lower')
    ax.set_xlabel('x')
    ax.set_ylabel('z')

    ax = fig.add_subplot(1, 3, 3)
    im = ax.imshow(result_yz, extent=[0, 1, 0, 1], origin='lower')
    ax.set_xlabel('y')
    ax.set_ylabel('z')
    
visualize(result_gpu, npoints)
visualize(result_cpu, npoints)

#rtol=1e-05, atol=1e-08
print("Do the results agree up to the defined tolerance? ",
      np.allclose(result_gpu, result_cpu))


The solution was computed using 400 points and 64 sources.

The cpu implementation uses double precision for the results but you can see the results still agree allowing the given tolerance due to different precision. The gpu kernel runs the computation using single precision numbers, which gives us some performance advantage due to its architecture.

### Benchmark results

The code below demonstrates how long it takes to run the cpu vs gpu computation using 800 points and 256 sources.


In [5]:
def generate_parameters(npoints, nsources):
    plot_grid = np.mgrid[0:1:npoints * 1j, 0:1:npoints * 1j]

    targets_xy = np.vstack((plot_grid[0].ravel(),
                            plot_grid[1].ravel(),
                            np.zeros(plot_grid[0].size))).T
    targets_xz = np.vstack((plot_grid[0].ravel(),
                            np.zeros(plot_grid[0].size),
                            plot_grid[1].ravel())).T
    targets_yz = np.vstack((np.zeros(plot_grid[0].size),
                            plot_grid[0].ravel(),
                            plot_grid[1].ravel())).T

    targets = np.vstack((targets_xy, targets_xz, targets_yz))
    result_cpu = np.zeros(len(targets), dtype=np.float64)
    rand = np.random.RandomState(0)

    sources = rand.rand(nsources, 3)
    weights = rand.rand(len(sources))
    return (targets, result_cpu, sources, weights)
    

In [6]:
sigma = .1
npoints = 800
nsources = 256

(targets, result_cpu, sources, weights)= generate_parameters(npoints, nsources)

sources_global_mem = cuda.to_device(sources.astype('float32'))
targets_global_mem = cuda.to_device(targets.astype('float32'))
weights_global_mem = cuda.to_device(weights.astype('float32'))
result_global_mem = cuda.device_array(len(targets), dtype=np.float32)
inter_result_global_mem = cuda.device_array((len(targets), p), dtype=np.float32)
l = (targets.shape[0] + SX - 1) // SX
p = (sources.shape[0] + SY - 1) // SY

In [7]:
%timeit rbf_evaluation_cpu(sources, targets, weights, result_cpu)

2.23 s ± 50.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit -n 20
rbf_evaluation_cuda[(l, p), (SX, SY)](sources_global_mem, targets_global_mem, weights_global_mem, inter_result_global_mem)
summation_kernel[(l, 1), (SX, 1)](inter_result_global_mem, result_global_mem,targets_global_mem)

Let's try to increase the number of sources and targets of the GPU computation:

In [ ]:
%%time
sigma = .1
npoints = 500
nsources = 512
(targets, result_cpu, sources, weights)= generate_parameters(npoints, nsources)
sources_global_mem = cuda.to_device(sources.astype('float32'))
targets_global_mem = cuda.to_device(targets.astype('float32'))
weights_global_mem = cuda.to_device(weights.astype('float32'))
result_global_mem = cuda.device_array(len(targets), dtype=np.float32)
l = (targets.shape[0] + SX - 1) // SX
p = (sources.shape[0] + SY - 1) // SY
result_gpu = result_global_mem.copy_to_host()

In [ ]:
%%timeit -n 20
rbf_evaluation_cuda[(l, p), (SX, SY)](sources_global_mem, targets_global_mem, weights_global_mem, inter_result_global_mem)
summation_kernel[(l, 1), (SX, 1)](inter_result_global_mem, result_global_mem,targets_global_mem)

We can see that the GPU computation has no problems to process 750000x3 targets and 512x3 sources. The bottleneck seems to be the cell that allocates and transfers the memory from CPU and GPU. Basically, almost all the time took to run the algorithm was just to allocate and transfer data from CPU to the GPU. 

Numba is not freeing up memory previously allocated in different cells since the context is being shared. It takes much longer to run the first time or second time. We can still use though the sys time to compare with the actual computation.

## Part 2 (Evaluating the discrete Laplacian on GPUs)

### CPU Implementation

In the lectures we have generated a sparse matrix which discretises Laplace operator with zero boundary conditions as follows:

In [9]:
N = 15
nelements = 5 * N**2 - 16 * N + 16
def discretise_poisson_cpu(N):
    """Generate the matrix and rhs associated with the discrete Poisson operator."""

    nelements = 5 * N**2 - 16 * N + 16
    row_ind = np.empty(nelements, dtype=np.float64)
    col_ind = np.empty(nelements, dtype=np.float64)
    data = np.zeros(nelements, dtype=np.float64)
    
    f = np.empty(N * N, dtype=np.float64)

    count = 0
    for j in range(N):
        for i in range(N):
            if i == 0 or i == N - 1 or j == 0 or j == N - 1:
                row_ind[count] = col_ind[count] = j * N + i
                data[count] = 1
                f[j * N + i] = 0
                count += 1

            else:
                row_ind[count: count + 5] = j * N + i
                col_ind[count] = j * N + i
                col_ind[count + 1] = j * N + i + 1
                col_ind[count + 2] = j * N + i - 1
                col_ind[count + 3] = (j + 1) * N + i
                col_ind[count + 4] = (j - 1) * N + i

                data[count] = 4 * (N - 1)**2
                data[count + 1: count + 5] = - (N - 1)**2
                f[j * N + i] = 1

                count += 5
    return coo_matrix((data, (row_ind, col_ind)), shape=(N**2, N**2)).tocsr(), f

### GPU Implementation
Create N² threads and given a one-dimensional array of values ui,j in the unit square grid evaluates this discrete Laplace operator without explicity creatiing a matrix

In [12]:
@cuda.jit
def evaluate_discrete_laplace(vec_in, vec_out, N):
        # Evaluate the discrete Laplace operator
        
        i, j = cuda.grid(2)
        
        if i >= N:
            return
        if j >= N:
            return
        
        # Compute the vector index
        k = j * N + i
        
        if i == 0 or i == N - 1 or j == 0 or j == N - 1:
            # We are at the boundary
            # Here, the matrix just acts like the identity
            vec_out[k] = vec_in[k]
            return
        
        # Now deal with the interior element
        
        up = vec_in[(j + 1) * N + i]
        down = vec_in[(j - 1) * N + i]
        left = vec_in[j * N + i - 1]
        right = vec_in[j * N + i + 1]
        center = vec_in[k]
        
        vec_out[k] = (N - 1)**2 * (numba.float32(4) * center - up - down - left - right)
        
def eval_gpu(x, N):
        #Evaluate the discrete Laplacian on the GPU.
        
        res = np.empty(N * N, dtype=np.float32)
        
        nblocks = (N + 31) // 32
        evaluate_discrete_laplace[(nblocks, nblocks), (32, 32)](x.astype('float32'), res, N)
        return res.astype('float64')

### Validation
Now let's validate our implementation.

In [13]:
rand = np.random.RandomState(0)

N = 500

A, _ = discretise_poisson_cpu(N)
x = rand.randn(N * N)

y_cpu = A @ x
y_gpu = eval_gpu(x, N)

rel_error = np.linalg.norm(y_cpu - y_gpu, np.inf) / np.linalg.norm(y_cpu, np.inf)
print(f"Relative errorr: {rel_error}.")

Relative errorr: 1.1318467088783189e-07.


The difference between the implementations is in the order of 32 bit machine precision, which is expected. Let us now do some benchmark comparisons.

### Benchmark results

The code below demonstrates how long it takes to run the cpu vs gpu computation having a grid of 2250000 points.

In [13]:
def benchmark(N):
    """Benchmark the CPU and GPU implementation."""
    
    A, _ = discretise_poisson_cpu(N)
    x = rand.randn(N * N)
    
    print("CPU Benchmark")
    %timeit y_cpu = A @ x
    
    print("GPU Benchmark")
    %timeit y_gpu = eval_gpu(x, N)

print("Small size benchmark (n=100)")
print("----------------------------")
benchmark(100)
print("Large size benchmark (n=1000)")
print("-------------------------------")
benchmark(1000)

Small size benchmark (n=100)
----------------------------
CPU Benchmark
38.7 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
GPU Benchmark
688 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Large size benchmark (n=1000)
-------------------------------
CPU Benchmark
4.83 ms ± 50.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
GPU Benchmark
4.04 ms ± 32.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Again, these benchmarks show that the GPU is faster for computing unknown points in a grid since we can load the memory only once and compute the five stencil points from the global memory. The radio of data will change based on the grid size, having bigger grids, we will have to move more interaction points from and to the CPU and GPU.

### How we could use shared memory to reduce global memory access

The difficult point of this problem is the 5 point stencil. Since the shared memory is only accessible within the block, we could have a local boundary point in a block that would not be a boundary value globally (ty = tx = 32, in  100x100 grid). So the solution to make this algorithm even quicker by using shared memory would be:

- load all grid points for that block when ty or tx are  0
- if tx is zero but px (global id) is not, load grid point for that block plus the previous column point grid_points(px−1,py)
- if ty is zero but py (global id) is not, load grid point for that block plus the previous row point grid_points(px,py-1)
- if tx is the last local thread for that block but px is not N-1, load grid point for that block plus the next column point grid_points(px+1,py)
- if ty is the last local thread for that block but py is not N-1, load grid point for that block plus the next row point grid_points(px,py +1)

Then we could read all 5 stencil points from the grid using the local shared memory and compute the approximated value. Each thread that is an interior point would write the approximation to the result array, which is allocated in the global memory.